In [1]:
pip install -U sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 47.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 42.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 50.6 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125926 sha256=11edf848161059b98e2a717bb1b75156cb48c3b82402cbff6f3989f9a3ee3c3e
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [2]:
from sklearn.model_selection import train_test_split
import numpy as np

In [3]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [4]:
df = pd.read_csv("/content/drive/MyDrive/aib/df.csv")

In [ ]:
df

In [6]:
n = 2300

In [ ]:
x = []
y = []

for i in range(n):
  if("(Novel)" in df.loc[i, "title"] or "(Light Novel)" in df.loc[i, "title"] or "(Webcomic)" in df.loc[i, "title"] or "(Doujinshi)" in df.loc[i, "title"]):
    continue
  if(df.loc[i, "q"]):
    print(i)
    x.append(df.loc[i, "q"].split('\n')[0])
    y.append(df.loc[i, "title"])

In [ ]:
x

In [8]:
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.2, random_state=42)

In [11]:
len(x_train)

1555

In [15]:
x_train = np.array(x_train)
x_val = np.array(x_val)

In [9]:
from sentence_transformers import SentenceTransformer, InputExample
from sentence_transformers import models, losses
from torch.utils.data import DataLoader

model_name = 'distilroberta-base'
word_embedding_model = models.Transformer(model_name, max_seq_length=32)
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())
model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

Some weights of the model checkpoint at distilroberta-base were not used when initializing RobertaModel: ['lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [10]:
train_data = [InputExample(texts=[s, s]) for s in x]
train_dataloader = DataLoader(train_data, batch_size=128, shuffle=True)
train_loss = losses.CosineSimilarityLoss(model)

In [23]:
print(train_data)

[<sentence_transformers.readers.InputExample.InputExample object at 0x7feba1e834f0>, <sentence_transformers.readers.InputExample.InputExample object at 0x7feba1e82980>, <sentence_transformers.readers.InputExample.InputExample object at 0x7feba1e83a90>, <sentence_transformers.readers.InputExample.InputExample object at 0x7feba1e82b30>, <sentence_transformers.readers.InputExample.InputExample object at 0x7feba1e81630>, <sentence_transformers.readers.InputExample.InputExample object at 0x7feba1e82a40>, <sentence_transformers.readers.InputExample.InputExample object at 0x7feba1e83f40>, <sentence_transformers.readers.InputExample.InputExample object at 0x7feba1e83370>, <sentence_transformers.readers.InputExample.InputExample object at 0x7feba1e839a0>, <sentence_transformers.readers.InputExample.InputExample object at 0x7feba1e82c80>, <sentence_transformers.readers.InputExample.InputExample object at 0x7feba1e829b0>, <sentence_transformers.readers.InputExample.InputExample object at 0x7feba1

In [11]:
model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs=1,
    show_progress_bar=True
)

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/16 [00:00<?, ?it/s]

RuntimeError: ignored